In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

## Initialize Tools

In [3]:
tools = [TavilySearchResults(max_results=1)]

## Create Agent

In [7]:
prompt = hub.pull("hwchase17/structured-chat-agent")

In [8]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format

In [9]:
llm = ChatOpenAI()

agent = create_structured_chat_agent(llm, tools, prompt)

## Run Agent

In [10]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [11]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "LangChain"
}
```[{'url': 'https://en.wikipedia.org/wiki/LangChain', 'content': 'In October 2023 LangChain introduced LangServe, a deployment tool designed to facilitate the transition from LCEL (LangChain Expression Language) prototypes to production-ready applications.[5]\nIntegrations[edit]\nAs of March 2023, LangChain included integrations with systems including Amazon, Google, and Microsoft Azure cloud storage; API wrappers for news, movie information, and weather; Bash for summarization, syntax and semantics checking, and execution of shell scripts; multiple web scraping subsystems and templates; few-shot learning prompt generation support; finding and summarizing "todo" tasks in code; Google Drive documents, spreadsheets, and presentations summarization, extraction, and creation; Google Search and Microsoft Bing web search; OpenAI, Anthropic, and Hugging Face languag

{'input': 'what is LangChain?',
 'output': 'LangChain is an open-source project launched in October 2022 by Harrison Chase. It is a language model integration framework with various integrations such as cloud storage, API wrappers, web scraping subsystems, and more. In April 2023, LangChain incorporated as a startup and received significant funding. It has gained popularity with contributions on GitHub, discussions on Twitter, and meetups in San Francisco and London. For more details, you can refer to the LangChain Wikipedia page.'}

Failed to patch https://api.smith.langchain.com/runs/4f8cebbe-eebd-46c4-b27e-724104764eab in LangSmith API. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/runs/4f8cebbe-eebd-46c4-b27e-724104764eab', '{"detail":"Payloads already received"}')
Connection error caused failure to patch https://api.smith.langchain.com/runs/65c0b254-d96d-490f-b1a1-4b70a7d8ae7a  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/65c0b254-d96d-490f-b1a1-4b70a7d8ae7a (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))"))
Failed to post https://api.smith.langchain.com/runs in LangSmith API. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/runs', '{"detail":"Payloads already received"}')


## Using with chat history

In [13]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
I can help with that! Let me look up your name for you.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {"query": "What is my name?"}
}
```[{'url': 'https://www.names.org/', 'content': 'Find out the meaning, origin, popularity, and trends of your name or any other name on Names.org. Search by name, letter, or category and discover the history and influence of your name.'}]```
{
  "action": "Final Answer",
  "action_input": "I'm sorry, I couldn't retrieve your name. Would you mind sharing it with me?"
}
```

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': "I'm sorry, I couldn't retrieve your name. Would you mind sharing it with me?"}